# Dev: Body TrackingのCSVデータを、 Unity用に変換

In [1]:
from pathlib import Path
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd

### const

In [2]:
KEY_TRANSLATOR_3DPOSE_TO_K4ABT = {
    0: {
        "name": "尻(尾てい骨)",
        "k4a_ID": 0,
        "k4a_name": "PELVIS",
    },
    1: {
        "name": "右尻(右足付け根)",
        "k4a_ID": 22,
        "k4a_name": "HIP_RIGHT",
    },
    2: {
        "name": "右ひざ",
        "k4a_ID": 23,
        "k4a_name": "KNEE_RIGHT",
    },
    3: {
        "name": "右足首",
        "k4a_ID": 24,
        "k4a_name": "ANKLE_RIGHT",
    },
    4: {
        "name": "左尻(左足付け根)",
        "k4a_ID": 18,
        "k4a_name": "HIP_LEFT",
    },
    5: {
        "name": "左ひざ",
        "k4a_ID": 19,
        "k4a_name": "KNEE_LEFT",
    },
    6: {
        "name": "左足首",
        "k4a_ID": 20,
        "k4a_name": "ANKLE_LEFT",
    },
    7: {
        "name": "脊椎",
        "k4a_ID": 1,
        "k4a_name": "SPINE_NAVAL",
    },
    8: {
        "name": "胸",
        "k4a_ID": 2,
        "k4a_name": "SPINE_CHEST",
    },
    9: {
        "name": "首/鼻",
        "k4a_ID": 3,
        "k4a_name": "NECK", 
    },
    10: {
        "name": "頭",
        "k4a_ID": 26,
        "k4a_name": "HEAD",
    },
    11: {
        "name": "左肩",
        "k4a_ID": 5,
        "k4a_name": "SHOULDER_LEFT",
    },
    12: {
        "name": "左ひじ",
        "k4a_ID": 6,
        "k4a_name": "ELBOW_LEFT",
    },
    13: {
        "name": "左手首",
        "k4a_ID": 7,
        "k4a_name": "WRIST_LEFT",
    },
    14: {
        "name": "右肩", 
        "k4a_ID": 12,
        "k4a_name": "SHOULDER_RIGHT",
    },
    15: {
        "name": "右ひじ",
        "k4a_ID": 13,
        "k4a_name": "ELBOW_RIGHT",
    },
    16: {
        "name": "右手首",
        "k4a_ID": 14,
        "k4a_name": "WRIST_RIGHT",
    },
}

In [3]:
KEY_TRANSLATOR_3DPOSE_TO_K4ABT 

{0: {'name': '尻(尾てい骨)', 'k4a_ID': 0, 'k4a_name': 'PELVIS'},
 1: {'name': '右尻(右足付け根)', 'k4a_ID': 22, 'k4a_name': 'HIP_RIGHT'},
 2: {'name': '右ひざ', 'k4a_ID': 23, 'k4a_name': 'KNEE_RIGHT'},
 3: {'name': '右足首', 'k4a_ID': 24, 'k4a_name': 'ANKLE_RIGHT'},
 4: {'name': '左尻(左足付け根)', 'k4a_ID': 18, 'k4a_name': 'HIP_LEFT'},
 5: {'name': '左ひざ', 'k4a_ID': 19, 'k4a_name': 'KNEE_LEFT'},
 6: {'name': '左足首', 'k4a_ID': 20, 'k4a_name': 'ANKLE_LEFT'},
 7: {'name': '脊椎', 'k4a_ID': 1, 'k4a_name': 'SPINE_NAVAL'},
 8: {'name': '胸', 'k4a_ID': 2, 'k4a_name': 'SPINE_CHEST'},
 9: {'name': '首/鼻', 'k4a_ID': 3, 'k4a_name': 'NECK'},
 10: {'name': '頭', 'k4a_ID': 26, 'k4a_name': 'HEAD'},
 11: {'name': '左肩', 'k4a_ID': 5, 'k4a_name': 'SHOULDER_LEFT'},
 12: {'name': '左ひじ', 'k4a_ID': 6, 'k4a_name': 'ELBOW_LEFT'},
 13: {'name': '左手首', 'k4a_ID': 7, 'k4a_name': 'WRIST_LEFT'},
 14: {'name': '右肩', 'k4a_ID': 12, 'k4a_name': 'SHOULDER_RIGHT'},
 15: {'name': '右ひじ', 'k4a_ID': 13, 'k4a_name': 'ELBOW_RIGHT'},
 16: {'name': '右手首', 'k4a

### functions

In [4]:
def to_unity_format(s):
    """Get a single row and convert it into unity format.
    Args:
        s (pd.Series): -
    Return:
        str
    """
    
    line = ""
    for joint_id in range(17):
        d = KEY_TRANSLATOR_3DPOSE_TO_K4ABT.get(joint_id)
        
        k4a_joint_id = d.get("k4a_ID")        
        px, py, pz =  s[f"J{k4a_joint_id:0=2}_P0"], s[f"J{k4a_joint_id:0=2}_P1"], s[f"J{k4a_joint_id:0=2}_P2"]
        
        line += f"{joint_id} {px} {-py} {pz}, " # Convert to left-handed system (Y-Up)
    return line

In [5]:
def select_body_index(df, index=0):
    """
    Note:
        複数のBodyが検出されているケースは、ここで消すべきだろうか？
    """
    df = df[df["body_index"] == 0].reset_index(drop=True)
    
    return df
    

def resample_to_15hz(df):
    df_new = df.copy()

    df["timestamp"] = pd.to_datetime(df["timestamp"], unit='s')
    df = df.set_index("timestamp").asfreq("66ms", method="ffill")
    
    return df.reset_index(drop=False)

In [6]:
def update_lower_body_key_points(df):
    lower_body_key_points = [
        19, 20, 21,
        23, 24, 25,
    ]
    
    for ind in lower_body_key_points:
        for axis in [0, 1, 2]:
            df[f"J{ind:0=2}_P{axis}"] = 0.0
            df[f"J{ind:0=2}_O{axis+1}"] = 0.0
            
        df[f"J{ind:0=2}_O0"] = 1.0
        
    return df


def update_out_of_range_key_points(df):
    NUM_K4ABT_JOINTS = 32
    K4ABT_OUT_OF_RANGE = 0
    
    # Replace OutOfRange data points with NaN
    for ind in range(NUM_K4ABT_JOINTS):
        cols = [
            f"J{ind:0=2}_P0", f"J{ind:0=2}_P1", f"J{ind:0=2}_P2",
            f"J{ind:0=2}_O1", f"J{ind:0=2}_O2", f"J{ind:0=2}_O3",
        ]
        
        ind_out = df[df[f"J{ind:0=2}_CONF"] == K4ABT_OUT_OF_RANGE].index
        df.loc[ind_out, cols] = 0.0
        df.loc[ind_out, f"J{ind:0=2}_O0"] = 0.0
        
        # # Fill NaN with method="ffill"
        # df[cols] = df[cols].fillna(method="ffill")
    return df


def apply_median_filter(df, win_size=5):
    """Apply Median filter to keypoints sequence.
    
    """
    NUM_K4ABT_JOINTS = 32
    K4ABT_OUT_OF_RANGE = 0
    K4ABT_LOW_CONF = 1
    EXCLUDE_JOINTS = (
        19, 20, 21,
        23, 24, 25,
    )
    
    cols = []
    for ind in range(NUM_K4ABT_JOINTS):
        if ind in EXCLUDE_JOINTS:
            continue   
        cols += [
            f"J{ind:0=2}_P0", f"J{ind:0=2}_P1", f"J{ind:0=2}_P2",
            f"J{ind:0=2}_O0", f"J{ind:0=2}_O1", f"J{ind:0=2}_O2", f"J{ind:0=2}_O3",
        ]
    
    df[cols] = df[cols].rolling(win_size, axis=0).median()
    return df

In [7]:
def write_pos_txt(df, fname):    
    with open(fname, 'w', encoding='utf-8', newline='\n') as f:
        for i in tqdm(range(len(df))):
            line = to_unity_format(df.loc[i, :])
            f.write(line + '\n')
    return True

### Conversion

In [20]:
VERSION = "v2-0-0"

input_file = "./data/U0002_S0100.csv"
output_file = f"./data/U0002_S0100_unity_{VERSION}.txt"

# Load File
df = pd.read_csv(input_file)
# display(df.head())

# Timestamp関連
df = select_body_index(df, index=0)
df = resample_to_15hz(df)

# Confidenceの扱い
df = update_lower_body_key_points(df)
df = update_out_of_range_key_points(df)

# Median Filterによる突発的なエラーの除去
df = apply_median_filter(df)

In [21]:
df.shape

(49720, 258)

In [22]:
df.head()

,timestamp,body_index,J00_P0,J00_P1,J00_P2,J00_O0,J00_O1,J00_O2,J00_O3,J00_CONF,...,J30_O3,J30_CONF,J31_P0,J31_P1,J31_P2,J31_O0,J31_O1,J31_O2,J31_O3,J31_CONF
0,2021-10-22 02:15:46.000406016,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,...,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
1,2021-10-22 02:15:46.066406016,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,...,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
2,2021-10-22 02:15:46.132406016,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,...,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
3,2021-10-22 02:15:46.198406016,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,...,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
4,2021-10-22 02:15:46.264406016,0,47.0129,351.886,2230.96,0.550014,-0.434476,0.473526,-0.532536,2,...,0.193177,2,-15.251,-198.904,2152.4,0.65355,-0.291262,0.606166,-0.380356,2


In [23]:
df.to_csv(f"./data/U0002_S0100_cleaned_{VERSION}.csv", index=False)

In [24]:
write_pos_txt(df, output_file)

  0%|          | 0/49720 [00:00<?, ?it/s]

True